In [4]:
# !pip install openmeteo_requests
# !pip install requests_cache
# !pip install retry
# !pip install retry_requests

In [7]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below

In [9]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-09-29",
	"end_date": "2025-10-13",
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "rain", "temperature_2m_spread", "cloud_cover_mid_spread", "wind_speed_10m_spread"],
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_temperature_2m_spread = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover_mid_spread = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_speed_10m_spread = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["rain"] = hourly_rain
hourly_data["temperature_2m_spread"] = hourly_temperature_2m_spread
hourly_data["cloud_cover_mid_spread"] = hourly_cloud_cover_mid_spread
hourly_data["wind_speed_10m_spread"] = hourly_wind_speed_10m_spread

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 52.5483283996582°N 13.407821655273438°E
Elevation: 38.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                          date  temperature_2m  relative_humidity_2m  \
0   2025-09-29 00:00:00+00:00          9.3085             84.668953   
1   2025-09-29 01:00:00+00:00          8.1085             86.008286   
2   2025-09-29 02:00:00+00:00          7.8085             87.477806   
3   2025-09-29 03:00:00+00:00          7.5585             88.673653   
4   2025-09-29 04:00:00+00:00          7.2585             90.196533   
..                        ...             ...                   ...   
355 2025-10-13 19:00:00+00:00          9.3585             82.663948   
356 2025-10-13 20:00:00+00:00          8.4585             84.573128   
357 2025-10-13 21:00:00+00:00          8.1085             85.415634   
358 2025-10-13 22:00:00+00:00          7.7085             86.564651   
359 2025-10-13 23:00:00+00:00          7.6085             88.371803   

     wind_speed_10m  rain